**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install yellowbrick
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

from class_utils.sklearn import (
    make_ext_column_transformer, transformer_extensions
)

import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist, squareform

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }

# create a directory for storing any outputs
import os
os.makedirs("data", exist_ok=True)
os.makedirs("output", exist_ok=True)

# create a synthetic dataset
_blobs, _labels = datasets.make_blobs(
    n_samples=600, random_state=3,
    cluster_std=0.75, centers=5
)

_df_blobs = pd.DataFrame(np.hstack([_blobs, _labels.reshape(-1, 1)]),
                         columns=['x', 'y', 'label'])
_df_blobs['y'] *= 100
_df_blobs.to_csv("data/blobs_2d.csv", index=False)

del _blobs
del _labels
del _df_blobs

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
cluster_colors = sns.color_palette()[1:]

def scatter_legend(ax, sc, labels, num_colors, color_array,
                   s, edgecolor):
    handles = []
    
    for i in range(num_colors):
        h = mlines.Line2D([0], [0], ls="", color=color_array[i],
                          ms=s, marker=sc.get_paths()[0],
                          markeredgecolor=edgecolor)
        handles.append(h)

    ax.legend(handles=handles, labels=labels)

def plot_data(
    data, cluster_centres=None, color='b', ax=None,
    cluster_colors=cluster_colors,
    edgecolors='k', labels=None,
    center_color='orange', center_size=200,
    legend=True
):
    if ax is None:
        ax = plt.gca()
        
    if labels is None:
        ax.scatter(data[:, 0], data[:, 1], s=50,
                   color=color, edgecolors=edgecolors)
    else:
        c = np.asarray(cluster_colors)[labels]
        
        sc = ax.scatter(data[:, 0], data[:, 1], s=50,
                        c=c, edgecolors=edgecolors,
                        #cmap=plt.cm.get_cmap('category10', np.max(labels)+1)
                       )
        
        if legend:
            nclusts = np.max(labels)+1
            scatter_legend(ax, sc, ['$c_{}$'.format(i) for i in range(nclusts)],
                           nclusts, cluster_colors, s=6, edgecolor='k')
        
    if not cluster_centres is None:
        ax.scatter(cluster_centres[:, 0],
                   cluster_centres[:, 1],
            s=center_size, c=center_color,
            edgecolors=edgecolors)

    ax.grid(ls='--')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_axisbelow(True)

## Hierarchické zhlukovanie pomocou SciPy

Ďalej sa pozrieme na praktický príklad aplikácie hierarchického zhlukovania na jednoduchá dátovú množinu 2D bodov. To nám umožní ilustrovať princípy a rozhrania a zároveň nám umožní ľahko porovnať výsledky zhlukovania s 2D grafom pôvodných dát.

### Načítanie a prespracovanie dátovej množiny

V prvom kroku načítame dáta z CSV súboru a predspracujeme ich. Budeme počítať vzdialenosti (v našom prípade euklidovské) a bude preto veľmi podstatné, aby sme dáta najprv štandardizovali.



In [ ]:
# we load the data from the CSV
df = pd.read_csv("data/blobs_2d.csv")

# all inputs are numeric
categorical_inputs = []
numeric_inputs = list(df.columns[:-1])

# the preprocessing pipeline
input_preproc = make_ext_column_transformer(
    (make_pipeline(
        transformer_extensions(
            SimpleImputer(strategy='constant', fill_value='MISSING')
        ),
        OneHotEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        transformer_extensions(
            SimpleImputer()
        ),
        StandardScaler()),
     numeric_inputs),

    return_dataframe=True,
    verbose_feature_names_out=False
)

# the preprocessed data and the classes
df_X = input_preproc.fit_transform(df[categorical_inputs+numeric_inputs])
X = df_X.values
labels = df["label"]

# let's also keep the unnormalized data
X_unnorm = df[categorical_inputs+numeric_inputs].values

# plot the data
plt.figure(figsize=(6, 5))
plot_data(X)

### Výpočet vzdialeností a linkage

Na realizáciu hierarchického zhlukovania budeme používať balíček `scipy.cluster.hierarchy` – skrátane `sch`. Mohli by sme namiesto neho použiť aj triedu `AgglomerativeClustering` z balíčka scikit-learn; implementácia v `scipy` je však lepšie intregrovaná s rôznymi vizualizačnými nástrojmi, ktoré budeme používať.

Rozhranie bude vyzerať trochu inak než sme si zvykli pri používaní balíčka scikit-learn, ale celý proces bude stále jednoduchý. Bude sa realizovať v dvoch krokoch:

# Najprv vypočítame vzdialenosti medzi všetkými pármi bodov pomocou funkcie `pdist` (pri použití predvolených argumentov sa vypočíta euklidovská vzdialenosť, sú však k dispozícii aj iné možnosti).
# Ďalej zostavíme linkage objekt pomocou funkcie `sch.linkage`, pričom špecifikujeme, aká linkage metóda sa má použiť. Tu použijeme metódu `ward`, ktorá sa snaží minimalizovať rozptyl vo novovzniknutých zhlukoch.
Získaný "linkage" objekt je štruktúra reprezentujúca hierarchické vzťahy medzi všetkými podzhlukmi identifikovanými v našich dátach.



In [ ]:
D = pdist(X)
L = sch.linkage(D, method='ward')

### Vykreslenie dendrogramu

Pomocou linkage objektu môžeme realizovať rôzne operácie, napr. vykresliť vizuálnu reprezentáciu hierarchických vzťahov, ktorá sa označuje ako **dendrogram** . Vykonať to môžeme volaním funkcie `sch.dendrogram`. Keďže pracujeme s pomerne veľkým počtom bodov, špecifikujeme pri volaní `no_labels=True` – popiskov by bolo toľko, že by boli tak či tak nečitateľné.

Keď si dendrogram vykreslíte a pozorne ho preskúmate, všimnete si, že výška jednotlivých vetiev v grafe je rôzna. Je to tak preto, lebo indikuje vzdialenosť medzi dvoma podzhlukmi, ktoré spája. Ako vidno, vzdialenosť (v zmysle zvolenej linkage metódy) medzi vysokoúrovňovými zhlukmi je omnoho väčšia než na spodnej úrovni stromu.



In [ ]:
G = sch.dendrogram(L, no_labels=True)
plt.xlabel("samples")
plt.ylabel("distance")

plt.savefig("output/hierarchical_scipy_dendrogram.svg", bbox_inches='tight', pad_inches=0)

Všimnite si, že keď realizujeme hierarchické zhlukovanie, výstupom nie je množina plochých (flat) zhlukov – výstupom zhlukovacieho algoritmu je linkage štruktúra, ktorú sme si práve vizualizovali. Získať množinu plochých zhlukov je však ľahké – stačí, aby sme si zvolili určitú prahovú úroveň, na ktorej dendrogram rozrežeme – body pod každou vetvou môžeme potom považovať za plochý zhluk. Jeden taký rez už ilustruje náš dendrogram – body pod ľavou vetvou sú zafarbené na oranžovo (zhluk 1) a body pod pravou vetvou zafarbené na zeleno (zhluk 2).

Skúsme si zvoliť inú prahovú úroveň, napr. `5` – získame inú množinu plochých zhlukov.



In [ ]:
G = sch.dendrogram(L, no_labels=True, color_threshold=5)
plt.xlabel("samples")
plt.ylabel("distance")

plt.savefig("output/hierarchical_scipy_dendrogram_thresh5.svg", bbox_inches='tight', pad_inches=0)

### Priradenie bodov do plochých zhlukov

Ak chceme získať priradenie každého bodu z našej dátovej množiny do plochého zhluku, môžeme použiť metódu `sch.fcluster`, pričom špecifikujeme linkage objekt, prahovú úroveň a kritérium (t.j. čo prahujeme). Ak chceme replikovať ploché zhluky, ktoré sme vizualizovali vyššie, špecifikujeme, že prahujeme `distance` (vzdialenosť) a prahovú úroveň nastavíme znovu na 5.

Ak sa chcete dozvedieť viac o ďalších kritériách, preštudujte si docstring funkcie `sch.fcluster` (napr. tak, že v bunke spustíte príkaz `?sch.fcluster`).

Všimnite si tiež, že funkcia `sch.fcluster` má drobnú zvláštnosť v tom, že **číslovanie zhlukov začína od 1** . Tu sa s tým vysporiadame tak, že od čísel **odčítame 1** , aby sme získali číslovanie začínajúce od 0).



In [ ]:
clusts = sch.fcluster(L, t=5, criterion='distance') - 1

Teraz si môžeme opäť vykresliť všetky pôvodné body a zafarbiť ich podľa vypočítaných čísel plochých zhlukov.



In [ ]:
plt.figure(figsize=(6, 5))
plot_data(X, labels=clusts)

### Vykreslenie heatmapy s dendrogramami

Ako sme už videli, hierchické zhlukovanie nám umožňuje vizualizovať si dendrogram a tým nám poskytuje silný nástroj na vizualizáciu štruktúry dátovej množiny. Ďalšia vizualizácia, ktorá vie byť veľmi užitočná, je hierarchicky zhlukovaná heatmapa.

Princíp spočíva v tom, že sa vypočítajú vzdialenosti medzi všetkými dvojicami bodov z dátovej množiny a vizualizujú sa vo forme heatmapy – zároveň sa však aplikuje hierarchické zhlukovanie na riadky aj stĺpce heatmapy, čo má za následok, že podobné body majú tendenciu sa v heatmape zoskupiť. Tento typ vizualizácie poskytuje ešte lepšiu predstavu o priestorovej štruktúre dátovej množinu než vie poskytnúť samotný dendrogram.

Aby bola vizualizácia robustnejšia, typicky sa pre riadky a pre stĺpce robí rozdielne zhlukovanie – tu budeme pre jedny používať `ward` linkage a pre druhé `single` linkage.



In [ ]:
L1 = sch.linkage(D, method='ward')
L2 = sch.linkage(D, method='single')

Existujú rôzne spôsoby ako realizovať samotné vykreslenie heatmapy, azda najjednoduchším je však využitie funkcie `clustermap` z balíčka `seaborn`. Stačí jej len postúpiť maticu vzdialeností a linkage objekty a funkcia sa postará o všetko ostatné.



In [ ]:
cg = sns.clustermap(squareform(D), row_linkage=L1, col_linkage=L2, rasterized=True)
cg.ax_cbar.set_ylabel("$L_2$ distance")
cg.ax_heatmap.set_xlabel("sample")
cg.ax_heatmap.set_ylabel("sample")

plt.savefig("output/hierarchical_dendrogram_heatmap.svg",
    bbox_inches='tight', pad_inches=0, dpi=300
)

### Vykreslenie zhlukovej mapy

Všimnite si, že aby sme vyššie vytvorili heatmapu, zavolali sme funkciu `clustermap` s maticou vzdialeností a s vopred vypočítanými linkage objektmi. Keď zavoláme tú istú funkciu priamo na pôvodnom dátovom rámci a s predvolenými argumentami, vytvorí iný typ grafu nazývaný zhluková mapa.

V tejto zhlukovej mape sú riadky zhlukované tak ako predtým, ale stĺpce teraz zodpovedajú stĺpcom z dátového rámca. V tomto syntetickom príklade pravdepodobne nebude hneď zrejmé, kde by mohla takáto vizualizácia byť užitočná. Uvedieme si preto aj malý príklad na inej dátovej množine, kde to vysvetlíme úplnejším spôsobom.



In [ ]:
cg = sns.clustermap(df_X)
cg.ax_cbar.set_ylabel("coordinate value")
cg.ax_heatmap.set_xlabel("coordinate")
cg.ax_heatmap.set_ylabel("sample")

### Zhluková mapa s otočenými dátami

Aby bolo možné lepšie porozumieť tomu, na čo môže poslúžiť zhluková mapa, odvoláme sa na príklad prezentovaný v [[medium_cluster_map]](#medium_cluster_map), ktorý využívať dátovú množinu "flights" dostupnú cez rozhranie balíčka `seaborn`. Načítajme si ju.



In [ ]:
df = sns.load_dataset('flights')
df.head()

Ako vidno, dátová množina má 3 stĺpce. Každý riadok indikuje počet leteckých pasažierov v danom mesiaci a danom roku. Dajme tomu, že chceme teraz preskúmať štruktúru týchto dát – konkrétne porozumieť, kedy sa najviac lieta, ktoré mesiace a roky sú si vzájomne podobné atď. Zdá sa, že ak to chceme urobiť, najlepšie by bolo dátovú množinu transformovať do maticového tvaru, kde každý riadok bude zodpovedať mesiacu, každú stĺpec roku a samotné hodnoty budú predstavovať počty pasažierov.

Tento typ reprezentácie vieme získať otočením (angl. pivoting) dátového rámca:



In [ ]:
df_pivot = df.pivot_table(index="month", columns="year", values="passengers")

#### Otočená zhluková mapa bez zhlukovania

Skúsme si teraz zobraziť zhlukovú mapu nášho otočeného dátového rámca (zatiaľ ešte bez zhlukovania). Čo vidíme okamžite je, že najviac sa lieta v júli a auguste a tiež, že v počte pasažierov bol za dané obdobie jasný rastúci trend.



In [ ]:
cg = sns.clustermap(df_pivot, col_cluster=False, row_cluster=False)
cg.ax_cbar.set_ylabel("passengers")

#### Otočená zhluková mapa so zhlukovaním

Dajme tomu, že teraz chceme lepšie porozumieť podobnostiam, medzi jednotlivými mesiacmi a rokmi. Stačí nám zavolať funkciu `clustermap` so zapnutým zhlukovaním (t.j. bez ďalších argumentov – len s dátovým rámcom).



In [ ]:
cg = sns.clustermap(df_pivot)
cg.ax_cbar.set_ylabel("passengers")

#### Otočená mapa zhlukov so zhlukovaním a štandardizáciou

Hoci vyššie zobrazená zhluková mapa nám dátva predstavu o tom, ktoré mesiace a roky sa na seba navzájom podobajú a aké medzi nimi z tohto pohľadu existujú skupiny, môžeme ísť ešte o krok ďalej.

Povedzme, že čomu chceme naozaj porozumieť je, ktoré mesiace sú si vzájomne podobné a aké skupiny existujú medzi nimi. V tom prípade budú meniace sa celkové počty pasažierov v jednotlivých rokoch tieto vzťahy zahmlievať. Aby sme tento vplyv eliminovali, môžeme si dáta pre každý rok preškálovať do intervalu od 0 po 1. Taká funkcionalita je už vstavaná, takže nám stačí pri volaní špecifikovať `standard_scale=1`. Keďže sa teraz už zaujímame len o vzájomné vzťahy medzi mesiacmi a nie medzdi rokmi, bude okrem toho asi dobrý nápad vypnúť zhlukovanie pre stĺpce a roky vykresliť v prirodzenom poradí: špecifikujeme teda `col_cluster=False`.

Teraz vidno vzťahy ešte jasnejšie. Je zrejmé, že vo všeobecnosti sa najviac lieta v letných mesiacoch – osobitne v júli a v auguste, ale v menšej miere aj v júni a v septembri. V rámci pozorovaného obdobia bolo omnoho menej pasažierov v zimných mesiacoch – v novembri, januári a februári; december bol výnimkou, pravdepdobne kvôli sviatkom. Zvyšné mesiace sú (spoločne s decembrom) kdesi uprostred.



In [ ]:
cg = sns.clustermap(df_pivot, standard_scale=1, col_cluster=False)
cg.ax_cbar.set_ylabel("passengers")

### References

<a id="medium_cluster_map">[medium_cluster_map]</a> Keith Brooks. Day (4) — Data Visualization — How to use Seaborn for Heatmaps. URL: <https://medium.com/@kbrook10/day-4-data-visualization-how-to-use-seaborn-for-heatmaps-bf8070e3846e>.

